In [6]:
pip install causal-learn


  Using cached causal_learn-0.1.3.8-py3-none-any.whl (174 kB)
  Using cached graphviz-0.20.3-py3-none-any.whl (47 kB)


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from causallearn.search.ConstraintBased.PC import pc
from causallearn.utils.GraphUtils import GraphUtils
from causallearn.utils.PCUtils.BackgroundKnowledge import BackgroundKnowledge
from causallearn.graph.GraphNode import GraphNode
import pydot
from IPython.display import Image, display

In [4]:
# gut microbes
gut_microbes = pd.read_csv("../datasets/gut_16s_abundance.txt", sep = "\t")
gut_microbes.head()

,SampleID,phylum_Actinobacteria,phylum_Bacteroidetes,phylum_Firmicutes,phylum_Proteobacteria,phylum_Verrucomicrobia,phylum_unclassified_Bacteria,class_Actinobacteria,class_Bacilli,class_Bacteroidia,...,genus_Veillonella,genus_unclassified_Bacteria,genus_unclassified_Clostridiales,genus_unclassified_Clostridiales_Incertae.Sedis.XIII,genus_unclassified_Coriobacteriaceae,genus_unclassified_Erysipelotrichaceae,genus_unclassified_Firmicutes,genus_unclassified_Lachnospiraceae,genus_unclassified_Porphyromonadaceae,genus_unclassified_Ruminococcaceae
0,ZOZOW1T-1010,0.000449,0.650866,0.225080,0.007364,0.015835,0.097431,0.000449,0.000138,0.650866,...,0.000000,0.097431,0.007883,0.000035,0.000415,0.002386,0.015697,0.027452,0.000000,0.074128
1,ZOZOW1T-1011,0.000176,0.730518,0.184808,0.003900,0.008186,0.070902,0.000176,0.000070,0.730518,...,0.000035,0.070902,0.008713,0.000141,0.000176,0.000808,0.008608,0.011805,0.000000,0.069215
2,ZOZOW1T-1012,0.000597,0.178364,0.796861,0.000358,0.002788,0.020911,0.000597,0.000119,0.178364,...,0.000080,0.020911,0.022903,0.000518,0.000398,0.001713,0.000876,0.018362,0.000000,0.615351
3,ZOZOW1T-1015,0.000058,0.740726,0.226516,0.005977,0.001207,0.024971,0.000058,0.000029,0.740726,...,0.000029,0.024971,0.013046,0.000086,0.000029,0.004109,0.007730,0.043849,0.000029,0.067096
4,ZOZOW1T-1021,0.001113,0.554580,0.364195,0.038358,0.000059,0.041286,0.001113,0.000059,0.554580,...,0.000410,0.041286,0.006208,0.000117,0.000176,0.000468,0.000996,0.132584,0.015753,0.044624


In [11]:
# gut_microbes.isnull().sum().sum() -> this is 0
gut_microbes_numeric = gut_microbes.drop(columns=['SampleID'])


In [14]:
# drop columns with very low variance
gut_microbes_numeric = gut_microbes_numeric.loc[:, gut_microbes_numeric.var() > 1e-5]
gut_microbes_numeric.head()

,phylum_Actinobacteria,phylum_Bacteroidetes,phylum_Firmicutes,phylum_Proteobacteria,phylum_Verrucomicrobia,phylum_unclassified_Bacteria,class_Actinobacteria,class_Bacilli,class_Bacteroidia,class_Betaproteobacteria,...,genus_Ruminococcus,genus_Streptococcus,genus_Veillonella,genus_unclassified_Bacteria,genus_unclassified_Clostridiales,genus_unclassified_Erysipelotrichaceae,genus_unclassified_Firmicutes,genus_unclassified_Lachnospiraceae,genus_unclassified_Porphyromonadaceae,genus_unclassified_Ruminococcaceae
0,0.000449,0.650866,0.225080,0.007364,0.015835,0.097431,0.000449,0.000138,0.650866,0.001936,...,0.0,0.000138,0.000000,0.097431,0.007883,0.002386,0.015697,0.027452,0.000000,0.074128
1,0.000176,0.730518,0.184808,0.003900,0.008186,0.070902,0.000176,0.000070,0.730518,0.001511,...,0.0,0.000035,0.000035,0.070902,0.008713,0.000808,0.008608,0.011805,0.000000,0.069215
2,0.000597,0.178364,0.796861,0.000358,0.002788,0.020911,0.000597,0.000119,0.178364,0.000080,...,0.0,0.000040,0.000080,0.020911,0.022903,0.001713,0.000876,0.018362,0.000000,0.615351
3,0.000058,0.740726,0.226516,0.005977,0.001207,0.024971,0.000058,0.000029,0.740726,0.001178,...,0.0,0.000029,0.000029,0.024971,0.013046,0.004109,0.007730,0.043849,0.000029,0.067096
4,0.001113,0.554580,0.364195,0.038358,0.000059,0.041286,0.001113,0.000059,0.554580,0.011595,...,0.0,0.000059,0.000410,0.041286,0.006208,0.000468,0.000996,0.132584,0.015753,0.044624


In [15]:
# removing highly correlated columns bc was giving error earlier 
# ValueError: Data correlation matrix is singular. Cannot run fisherz test. Please check your data.

corr_matrix = gut_microbes_numeric.corr().abs()

# upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# find columns with correlation greater than a threshold (e.g., 0.95)
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
gut_microbes_numeric = gut_microbes_numeric.drop(columns=to_drop)


In [17]:
# data input for FCI must be numpy array
data = np.array(gut_microbes_numeric)
#data

array([[4.49469000e-04, 6.50866093e-01, 2.25080386e-01, ...,
        2.74522010e-02, 0.00000000e+00, 7.41278570e-02],
       [1.75673000e-04, 7.30517883e-01, 1.84807814e-01, ...,
        1.18052140e-02, 0.00000000e+00, 6.92150940e-02],
       [5.97467000e-04, 1.78363738e-01, 7.96861308e-01, ...,
        1.83621450e-02, 0.00000000e+00, 6.15350912e-01],
       ...,
       [9.66183600e-03, 5.32155797e-01, 4.45727657e-01, ...,
        3.05706520e-02, 1.82669080e-02, 3.58544690e-02],
       [2.60721800e-03, 7.76423717e-01, 2.04212562e-01, ...,
        9.78439200e-03, 1.72838100e-03, 2.16486990e-02],
       [3.74855510e-02, 3.52837563e-01, 5.84466340e-01, ...,
        7.37601140e-02, 1.21318875e-01, 3.25865580e-02]])

In [20]:
from causallearn.search.ConstraintBased.FCI import fci
from causallearn.utils.cit import fisherz  # import the independence test method 

independent_test_method = fisherz

# default parameters
g, edges = fci(data)

# or customized parameters
g, edges = fci(data, independence_test_method, alpha, depth, max_path_length,
    verbose, background_knowledge, cache_variables_map)

# visualization
from causallearn.utils.GraphUtils import GraphUtils

pdy = GraphUtils.to_pydot(g)
pdy.write_png('simple_test.png')


  0%|          | 0/47 [00:00<?, ?it/s]

X3 --> X2
X2 --> X15
X6 --> X12
X25 --> X6
X13 --> X7
X9 --> X30
X9 --> X31
X44 --> X9
X23 --> X13
X37 --> X21
X37 --> X22
X43 --> X24
X25 --> X47
X29 --> X34
X34 --> X39


NameError: name 'independence_test_method' is not defined